In [1]:
import pandas as pd
import ast
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
load = 'eth'
df = pd.read_csv(f'data/orderbook_{load}_usdt.csv', parse_dates=['date'])
df['asks'] =  df['asks'].map(lambda d : ast.literal_eval(d))
df['bids'] =  df['bids'].map(lambda d : ast.literal_eval(d))

In [3]:
backup = df

In [4]:
specs = { #initial margin , maintenance margin rate
  'eth': (np.round(100000/100/100000,4), 0.005),
  'btc': (np.round(50000/125/50000,4), 0.004),
  'doge': (np.round(50000/50/50000,4), 0.01),
  'bnb': (np.round(10000/75/10000,4), 0.0065),
  'ada': (np.round(10000/75/10000,4), 0.0065),
}
specs

{'eth': (0.01, 0.005),
 'btc': (0.008, 0.004),
 'doge': (0.02, 0.01),
 'bnb': (0.0133, 0.0065),
 'ada': (0.0133, 0.0065)}

In [14]:
interest_index = 0.0001
imn = 200/specs['eth'][0] #(impact margin notional = 200 usdt / initial margin rate at maximum leverage level); 10% for doge

df['impact_bid'] = 0.0
df['impact_ask'] = 0.0

In [ ]:
for i in range(0, len(df)):
    df['impact_bid'][i], df_temp = calc_impact_bid(imn, i)
    df['impact_ask'][i], df_temp = calc_impact_ask(imn, i)


In [ ]:
df['premium_index'] = 0.0
#mid price is technically not correct: https://www.binance.com/en/futures/funding-history/3
df['premium_index'] = [(max(0, df['impact_bid'][i] - df['mid_price'][i]) - max(0, df['mid_price'][i] - df['impact_ask'][i])) / df['mid_price'][i] for i in df['premium_index']]

In [ ]:
weights = np.arange(1,2*60*8+1,1) #weighted moving average for 8hours
df['wma_premium_index'] = df['premium_index'].rolling(2*60*8).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
df['wma_premium_index'] = df['wma_premium_index'].fillna(0)

In [ ]:
df['funding_rate'] = df['wma_premium_index'] + interest_index

#clamp funding rate
df['funding_rate'][(df['wma_premium_index'] > -.0004) & (df['wma_premium_index'] < 0.0006)] = interest_index

#clamp funding rate to min and max
mmr = 0.005
floor = -0.75 * mmr
cap = 0.75 * mmr

df['funding_rate'] = [max(floor, min(i, cap))  for i in df['funding_rate']]

In [6]:
def calc_impact_bid(imn, i):
    df_temp = pd.DataFrame.from_dict(df['bids'][i], dtype='float')
    df_temp.sort_values(by='price', ascending=False, ignore_index=True, inplace=True)
    df_temp['cum_base_quantity'] = df_temp['amount'].cumsum()
    df_temp['quote_notional'] = df_temp['amount'] * df_temp['price']
    df_temp['cum_quote_notional'] = df_temp['quote_notional'].cumsum()
    
    df_temp['cum_quote_notional'].iloc[-1] += imn #hacky way of ensuring, that the last level is sufficient to cover imn
    
    idx = df_temp[df_temp['cum_quote_notional'] > imn].index[0] #determine which level is sufficient for imn
    if idx == 0:
        impact_bid = df_temp['price'][idx]
    else:
        impact_bid = imn / ((imn - df_temp['cum_quote_notional'][idx-1])/df_temp['price'][idx] + df_temp['cum_base_quantity'][idx-1])
    
    return impact_bid, df_temp

In [7]:
def calc_impact_ask(imn, i):
    df_temp = pd.DataFrame.from_dict(df['asks'][i], dtype='float')
    df_temp.sort_values(by='price', ascending=True, ignore_index=True, inplace=True)
    df_temp['cum_base_quantity'] = df_temp['amount'].cumsum()
    df_temp['quote_notional'] = df_temp['amount'] * df_temp['price']
    df_temp['cum_quote_notional'] = df_temp['quote_notional'].cumsum()
    
    df_temp['cum_quote_notional'].iloc[-1] += imn

    idx = df_temp[df_temp['cum_quote_notional'] > imn].index[0] #determine which level is sufficient for imn
    if idx == 0:
        impact_ask = df_temp['price'][idx]
    else:
        impact_ask = imn / ((imn - df_temp['cum_quote_notional'][idx-1])/df_temp['price'][idx] + df_temp['cum_base_quantity'][idx-1])
    
    return impact_ask, df_temp